In [1]:
from matplotlib import pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from sklearn import metrics, utils
from joblib import Parallel, delayed
from tqdm import tqdm#_notebook as tqdm
import scipy
import scipy.special
import itertools
import yaml
import pickle

In [2]:
path = './'

In [3]:
def get_ys(model_name, outcome):
    y_true = np.load(path + outcome + '_test.npy')
    y_score = None
    if model_name == 'xgb' or model_name == 'linear_svc':
        y_score = np.load(path + model_name + '_' + outcome + '_y_score.npy')
    else:
        y_score = np.load(path + model_name + '_' + outcome + '_y_score.npz')['y_score']      
    return y_true, y_score

In [47]:
def boostrap_func_all(i, y_true, y_prob, threshold):
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_prob)
    y_true_b, y_prob_b = utils.resample(y_true, y_prob, replace=True, random_state=i)
    y_pred_b = (y_prob_b > threshold)
    tpr_cutoff = metrics.recall_score(y_true_b, y_pred_b)
    idx = (np.abs(tpr - tpr_cutoff)).argmin()
    
    return (
        metrics.roc_auc_score(y_true_b, y_prob_b), # AUC
        tpr[idx], # sensitivity
        1-fpr[idx], # specificity
        metrics.precision_score(y_true_b, y_pred_b), # positive predictive value
    )

def boostrap_func_confusion(i, y_true, y_prob, threshold):
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_prob)
    y_true_b, y_prob_b = utils.resample(y_true, y_prob, replace=True, random_state=i)
    y_pred_b = (y_prob_b > threshold)
    tpr_cutoff = metrics.recall_score(y_true_b, y_pred_b)
    idx = (np.abs(tpr - tpr_cutoff)).argmin()
    
    return metrics.confusion_matrix(y_true_b, y_pred_b).ravel()

In [65]:
model = 'linear_svc'
task = 'refill'

In [66]:
y_true, y_score = get_ys(model, task)
fpr, tpr, thresh = metrics.roc_curve(y_true, y_score)
prec, rec, thresholds = metrics.precision_recall_curve(y_true, y_score)

max_rec = rec[prec >= 0.2].max()
select_thresh = thresholds[prec[:-1] >= 0.2][rec[:-1][prec[:-1] >= 0.2].argmax()]
select_idx = np.argmin(np.abs(thresholds-select_thresh))
select_prec = prec[select_idx]
select_spec = 1 - fpr[np.argmin(np.abs(tpr-max_rec))]
print('Thresh', (select_idx, select_thresh), 'Prec={:.3f}'.format(select_prec), ' Rec={:.3f}'.format(max_rec), ' Spec={:.3f}'.format(select_spec))

print('scores (95%CI lower, upper)')
auc_scores, sensitivities, specificities, ppvs = zip(*Parallel(n_jobs=4)(delayed(boostrap_func_all)(i, y_true, y_score, select_thresh) for i in range(1000)))
print('Test AUC {:.3f} ({:.3f}, {:.3f})'.format(np.mean(auc_scores), np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))
print('sens.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(sensitivities), np.percentile(sensitivities, 2.5), np.percentile(sensitivities, 97.5)))
print('spec.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(specificities), np.percentile(specificities, 2.5), np.percentile(specificities, 97.5)))
print('prec.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(ppvs), np.percentile(ppvs, 2.5), np.percentile(ppvs, 97.5)))

Thresh (18228, -0.6528159803079673) Prec=0.200  Rec=0.360  Spec=0.833
scores (95%CI lower, upper)
Test AUC 0.666 (0.654, 0.678)
sens.	 36.0% (34.2%, 38.0%)
spec.	 83.2% (82.0%, 84.1%)
prec.	 20.0% (18.8%, 21.2%)


In [67]:
model = 'xgb'
task = 'refill'

In [68]:
y_true, y_score = get_ys(model, task)
fpr, tpr, thresh = metrics.roc_curve(y_true, y_score)
prec, rec, thresholds = metrics.precision_recall_curve(y_true, y_score)

max_rec = rec[prec >= 0.2].max()
select_thresh = thresholds[prec[:-1] >= 0.2][rec[:-1][prec[:-1] >= 0.2].argmax()]
select_idx = np.argmin(np.abs(thresholds-select_thresh))
select_prec = prec[select_idx]
select_spec = 1 - fpr[np.argmin(np.abs(tpr-max_rec))]
print('Thresh', (select_idx, select_thresh), 'Prec={:.3f}'.format(select_prec), ' Rec={:.3f}'.format(max_rec), ' Spec={:.3f}'.format(select_spec))

print('scores (95%CI lower, upper)')
auc_scores, sensitivities, specificities, ppvs = zip(*Parallel(n_jobs=4)(delayed(boostrap_func_all)(i, y_true, y_score, select_thresh) for i in range(1000)))
print('Test AUC {:.3f} ({:.3f}, {:.3f})'.format(np.mean(auc_scores), np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))
print('sens.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(sensitivities), np.percentile(sensitivities, 2.5), np.percentile(sensitivities, 97.5)))
print('spec.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(specificities), np.percentile(specificities, 2.5), np.percentile(specificities, 97.5)))
print('prec.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(ppvs), np.percentile(ppvs, 2.5), np.percentile(ppvs, 97.5)))

Thresh (16919, 0.25270462) Prec=0.200  Rec=0.464  Spec=0.785
scores (95%CI lower, upper)
Test AUC 0.681 (0.669, 0.693)
sens.	 46.5% (44.5%, 48.5%)
spec.	 78.3% (76.5%, 79.8%)
prec.	 20.0% (19.0%, 21.1%)


In [69]:
model = 'linear_svc'
task = 'prolonged_use'

In [70]:
y_true, y_score = get_ys(model, task)
fpr, tpr, thresh = metrics.roc_curve(y_true, y_score)
prec, rec, thresholds = metrics.precision_recall_curve(y_true, y_score)

max_rec = rec[prec >= 0.2].max()
select_thresh = thresholds[prec[:-1] >= 0.2][rec[:-1][prec[:-1] >= 0.2].argmax()]
select_idx = np.argmin(np.abs(thresholds-select_thresh))
select_prec = prec[select_idx]
select_spec = 1 - fpr[np.argmin(np.abs(tpr-max_rec))]
print('Thresh', (select_idx, select_thresh), 'Prec={:.3f}'.format(select_prec), ' Rec={:.3f}'.format(max_rec), ' Spec={:.3f}'.format(select_spec))

print('scores (95%CI lower, upper)')
auc_scores, sensitivities, specificities, ppvs = zip(*Parallel(n_jobs=4)(delayed(boostrap_func_all)(i, y_true, y_score, select_thresh) for i in range(1000)))
print('Test AUC {:.3f} ({:.3f}, {:.3f})'.format(np.mean(auc_scores), np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))
print('sens.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(sensitivities), np.percentile(sensitivities, 2.5), np.percentile(sensitivities, 97.5)))
print('spec.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(specificities), np.percentile(specificities, 2.5), np.percentile(specificities, 97.5)))
print('prec.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(ppvs), np.percentile(ppvs, 2.5), np.percentile(ppvs, 97.5)))

Thresh (22280, -0.6726043713977421) Prec=0.200  Rec=0.042  Spec=0.991
scores (95%CI lower, upper)
Test AUC 0.660 (0.643, 0.675)
sens.	 4.1% (3.0%, 5.4%)
spec.	 99.1% (98.7%, 99.4%)
prec.	 20.1% (15.1%, 25.4%)


In [73]:
model = 'xgb'
task = 'prolonged_use'

In [74]:
y_true, y_score = get_ys(model, task)
fpr, tpr, thresh = metrics.roc_curve(y_true, y_score)
prec, rec, thresholds = metrics.precision_recall_curve(y_true, y_score)

max_rec = rec[prec >= 0.2].max()
select_thresh = thresholds[prec[:-1] >= 0.2][rec[:-1][prec[:-1] >= 0.2].argmax()]
select_idx = np.argmin(np.abs(thresholds-select_thresh))
select_prec = prec[select_idx]
select_spec = 1 - fpr[np.argmin(np.abs(tpr-max_rec))]
print('Thresh', (select_idx, select_thresh), 'Prec={:.3f}'.format(select_prec), ' Rec={:.3f}'.format(max_rec), ' Spec={:.3f}'.format(select_spec))

print('scores (95%CI lower, upper)')
auc_scores, sensitivities, specificities, ppvs = zip(*Parallel(n_jobs=4)(delayed(boostrap_func_all)(i, y_true, y_score, select_thresh) for i in range(1000)))
print('Test AUC {:.3f} ({:.3f}, {:.3f})'.format(np.mean(auc_scores), np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))
print('sens.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(sensitivities), np.percentile(sensitivities, 2.5), np.percentile(sensitivities, 97.5)))
print('spec.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(specificities), np.percentile(specificities, 2.5), np.percentile(specificities, 97.5)))
print('prec.\t {:.1%} ({:.1%}, {:.1%})'.format(np.mean(ppvs), np.percentile(ppvs, 2.5), np.percentile(ppvs, 97.5)))

Thresh (22017, 0.20804794) Prec=0.200  Rec=0.036  Spec=0.992
scores (95%CI lower, upper)
Test AUC 0.659 (0.642, 0.675)
sens.	 3.6% (2.6%, 4.7%)
spec.	 99.2% (98.9%, 99.5%)
prec.	 20.0% (15.0%, 25.6%)
